# Generate Company's Brochure with Multi-shot Prompting

In [1]:
# importing necessary libraries
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to Parse webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


In [10]:
conti = Website("https://www.continental-automotive.com/en.html")
#conti.links

# Let's use the LLMs to find relevant Links using Multi-shot Prompting 

In [7]:
system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
system_prompt += "You should respond in JSON as in this example:"
system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}\n
"""
system_prompt += "You should definitely include at least one link relating to media websites like Youtube/LinkedIn if available. \
You should respond in JSON as in this example:"
system_prompt += """
{
    "links": [
        {"type": "social media", "url": "https://full.url/goes/here/about/media"},
    ]
}
"""

In [8]:
print(system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}

You should definitely include at least one link relating to media websites like Youtube/LinkedIn if available. You should respond in JSON as in this example:
{
    "links": [
        {"type": "social media", "url": "https://full.url/goes/here/about/media"},
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
print(get_links_user_prompt(conti))

Here is the list of links on the website of https://www.continental-automotive.com/en.html - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://www.continental-automotive.com/en.html
https://www.continental-automotive.com/en/news.html
https://www.continental-automotive.com/en/company.html
https://www.continental.com/en/press/press-releases/
https://www.continental-automotive.com/en/general-information/metanav/Career.html
https://www.continental-automotive.com/en/focus-topics.html
https://www.continental-automotive.com/en/focus-topics/software-defined-vehicle.html
https://www.continental-automotive.com/en/focus-topics/exciting-mobility.html
https://www.continental-automotive.com/en/focus-topics/sustainable-mobility.html
https://www.continental-automotive.com/en/focus-topics/safe-mobility.html
https://www.c

# Calling the API

In [18]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)

In [19]:
# Call
get_links("https://www.continental-automotive.com/en.html")

{'links': [{'type': 'about page',
   'url': 'https://www.continental-automotive.com/en/company.html'},
  {'type': 'careers page',
   'url': 'https://www.continental-automotive.com/en/general-information/metanav/Career.html'},
  {'type': 'social media',
   'url': 'https://www.linkedin.com/company/continental/'},
  {'type': 'news page',
   'url': 'https://www.continental-automotive.com/en/news.html'}]}

# Making second API call
We will now use the contents of the first call to make a second call to make the brochure

In [20]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [25]:
#print(get_all_details("https://www.continental-automotive.com/en.html"))

In [23]:
system_prompt_brochure = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
If you have information about company's social media, emphasize it in the brochure with emojis of that social media platform."

In [24]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 5,000 characters
    return user_prompt

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_brochure},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [31]:
create_brochure("Continental Automotive", "https://www.continental-automotive.com/en.html")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.continental-automotive.com/en/company.html'}, {'type': 'careers page', 'url': 'https://www.continental-automotive.com/en/general-information/metanav/Career.html'}, {'type': 'news page', 'url': 'https://www.continental-automotive.com/en/news.html'}, {'type': 'social media', 'url': 'https://www.linkedin.com/company/continental/'}, {'type': 'social media', 'url': 'https://www.youtube.com/@TheAutomotiveGroup'}]}


# Continental Automotive Brochure

---

## Welcome to Continental Automotive

At **Continental Automotive**, we are dedicated to shaping the future of mobility through innovative technologies in passive safety, braking systems, automated driving, and connectivity that enhance every driving experience. As we transition towards becoming the independent automotive powerhouse **Aumovio**, our focus remains on **software-defined vehicles** and sustainable mobility solutions.

---

## Our Focus Topics

- **Software-defined Vehicle**
- **Exciting Driving Experience**
- **Sustainable Mobility**
- **Safe Mobility**
- **Electric and Autonomous Mobility**

---

## Our Solutions

### Diverse Offerings for All Vehicle Types

1. **Passenger Cars**
2. **Trucks & Buses**
3. **Two-Wheelers**
4. **Agriculture & Construction**
5. **Material Handling & Railway Solutions**

We create robust, high-tech components including advanced **brake systems**, **sensor technologies**, **display solutions**, and comprehensive **connectivity options**.

---

## Company Culture

At Continental Automotive, we foster a culture of **innovation, collaboration, and sustainability**. Our team is committed to improving mobility safety and efficiency, and we encourage a work environment where creativity thrives. Our workforce is empowered to leverage artificial intelligence and cutting-edge technologies, making us leaders in the automotive sector.

### Careers at Continental Automotive

Join a team dedicated to leading automotive technology innovations. We are always on the lookout for talented individuals ready to shape the future of mobility. Explore our career opportunities [here](#).

---

## Our Customers

We serve a broad spectrum of clients, including automotive manufacturers, fleet operators, and commercial vehicle manufacturers. Through strategic partnerships, such as our collaboration with **Aurora** for autonomous trucking solutions, we work towards providing tailored technologies for our diverse clientele.

---

## Stay Connected

Follow us on our social media platforms to stay updated on our latest innovations and technologies! 

- **Twitter**: [@Continental](https://twitter.com/CONTITECH) 🐦
- **LinkedIn**: [Continental Automotive LinkedIn](https://www.linkedin.com/company/continental/) 💼
- **Facebook**: [Continental Automotive Facebook](https://www.facebook.com/Continental/) 📘
- **Instagram**: [Continental Automotive Instagram](https://www.instagram.com/continental/) 📷

---

## Join Us on Our Journey

Ready to be part of the automotive revolution? Whether you're a customer, investor, or prospective recruit, Continental Automotive invites you to engage with our cutting-edge technologies and visionary approach to mobility. 

**Contact Us**: [Visit our website](https://www.continental.com).

--- 

Thank you for exploring Continental Automotive! Together, we drive the future of mobility.

# Streaming in contents instead of block

In [32]:
# Streaming the contents
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_brochure},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [33]:
stream_brochure("Continental Automotive", "https://www.continental-automotive.com/en.html")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.continental-automotive.com/en/company.html'}, {'type': 'careers page', 'url': 'https://www.continental-automotive.com/en/general-information/metanav/Career.html'}, {'type': 'social media', 'url': 'https://www.linkedin.com/company/continental/'}, {'type': 'social media', 'url': 'https://www.youtube.com/@TheAutomotiveGroup'}]}


# Welcome to Continental Automotive! 🚗

At **Continental Automotive**, we're transforming mobility through cutting-edge technologies and innovative solutions. As a leading partner in the automotive industry, we focus on making vehicles safer, greener, and more enjoyable to drive.

---

## **About Us 🌍**

Continental Automotive specializes in technologies for:

- **Passive Safety & Braking Systems**
- **Chassis Solutions & Motion Control**
- **Assisted & Automated Driving Technologies**
- **Advanced Display & Infotainment Systems**
- **Vehicle Electronics & High-Performance Compute Solutions**

### **Our Vision**
We’re driving towards a future of **Sustainable Mobility, Safe Mobility**, and the **Software-defined Vehicle** to ensure our customers enjoy an exciting driving experience.

---

## **Our Culture 💼**

At Continental, we foster an inclusive and innovative culture. Our teams are empowered to share ideas and grow professionally. We are committed to employee development, incorporating **Artificial Intelligence** initiatives, and participating in partnerships for continuous learning.

- **Collaborative Environment**: We value teamwork and collaboration across departments.
- **Commitment to Safety**: Grounded in our **Vision Zero** initiative, we aim for zero accidents and injuries.
- **Emphasis on Sustainability**: We focus on electric and eco-friendly mobility solutions.

---

## **Our Customers 👥**

We proudly serve a diverse range of customers across multiple industries, including:

- **Passenger Cars** 🚗
- **Trucks & Buses** 🚚
- **Two-Wheelers** 🏍️
- **Agriculture & Construction** 🚜
- **Railways** 🚆

With advanced technologies tailored for each segment, we ensure optimal performance and safety in every vehicle.

---

## **Join Our Team! 🚀**

**Careers at Continental Automotive:**  
We’re always looking for passionate problem solvers to join our talented workforce. Our career opportunities span across various disciplines, from engineering to marketing.

- **Explore Open Positions**: Check out our career page for opportunities that match your skills and aspirations!
- **Student Programs**: We offer internships and co-op programs for students looking to gain hands-on experience.

---

## **Stay Connected! 📱**

Follow us on our social media platforms to keep up with the latest news, innovations, and career opportunities:

- [Twitter](https://twitter.com/continental) 🐦
- [LinkedIn](https://linkedin.com/company/continental) 💼
- [Facebook](https://facebook.com/continental) 📘

---

Thank you for considering **Continental Automotive**. Together, let's drive into the future of mobility!

# Using Ollama for Brochure
We are using llama3.2 and it was observed that it could not produce satisfying results for some complex websites

In [45]:
#Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [80]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

# Keeping system prompt single shot for llama3.2 capabilities --> 1st Call
system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
system_prompt += "You should respond in JSON as in this example:"
system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""
# 2nd Call
system_prompt_brochure = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information.\
If you have information about company's social media, emphasize it in the brochure with emojis of that social media platform."

#Helper functions
def get_links_ollama(url, model="llama3.2"):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = model,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    return json.loads(response.choices[0].message.content)


def get_all_details_ollama(url, model="llama3.2"):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links_ollama(url, model)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

def get_brochure_user_prompt_ollama(company_name, url, model="llama3.2"):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details_ollama(url, model)
    user_prompt = user_prompt[:20_000] # Truncate if more than 5,000 characters
    return user_prompt

def create_brochure_ollama(company_name, url, model="llama3.2"):
    response = ollama_via_openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_brochure},
            {"role": "user", "content": get_brochure_user_prompt_ollama(company_name, url, model)}
          ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

# Streaming the contents
def stream_brochure_ollama(company_name, url, model="llama3.2"):
    stream = ollama_via_openai.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt_brochure},
            {"role": "user", "content": get_brochure_user_prompt_ollama(company_name, url, model)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [63]:
# Pull the model
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [75]:
med = Website("https://vercel.com/")
med.title

'Vercel: Build and deploy the best web experiences with the Frontend Cloud'

In [76]:
get_brochure_user_prompt_ollama("Vercel", "https://vercel.com/")

Found links: {'links': [{'type': 'home', 'url': 'https://vercel.com/'}, {'type': 'about', 'url': 'https://vercel.com/about'}, {'type': 'blog', 'url': 'https://vercel.com/blog'}, {'type': 'careers', 'url': 'https://vercel.com/careers'}, {'type': 'contact', 'url': 'https://vercel.com/contact'}, {'type': 'docs', 'url': 'https://vercel.com/docs'}, {'type': 'features.edge-functions', 'url': 'https://vercel.com/enterprise/features/edge-functions'}, {'type': 'legal/privacy-policy', 'url': 'https://vercel.com/legal/privacy-policy'}, {'type': 'marketplace', 'url': 'https://vercel.com/marketplace'}, {'type': 'pricing', 'url': 'https://vercel.com/pricing'}]}


'You are looking at a company called: Vercel\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nVercel: Build and deploy the best web experiences with the Frontend Cloud\nWebpage Contents:\nProducts\nDX Platform\nPreviews\nHelping teams ship 6× faster\nAI\nPowering breakthroughs\nManaged Infrastructure\nFluid compute\nServers, in serverless form\nRendering\nFast, scalable, and reliable\nObservability\nTrace every step\nSecurity\nScale without compromising\nOpen Source\nNext.js\nThe native Next.js platform\nTurborepo\nSpeed with Enterprise scale\nAI SDK\nThe AI Toolkit for TypeScript\nSolutions\nUse Cases\nAI Apps\nDeploy at the speed of AI\nComposable Commerce\nPower storefronts that convert\nMarketing Sites\nLaunch campaigns fast\nMulti-tenant Platforms\nScale apps with one codebase\nWeb Apps\nShip features, not infrastructure\nUsers\nPlatform Engineers\nAutomat

In [77]:
create_brochure_ollama("Vercel", "https://vercel.com/", "llama3.2")

Found links: {'links': [{'type': 'about page', 'url': 'https://vercel.com/about'}, {'type': 'blog', 'url': 'https://vercel.com/blog'}, {'type': 'changelog', 'url': 'https://vercel.com/changelog'}, {'type': 'careers', 'url': 'https://vercel.com/careers'}, {'type': 'contact page', 'url': 'https://vercel.com/contact'}, {'type': 'customers page', 'url': 'https://vercel.com/customers'}, {'type': 'docs page', 'url': 'https://vercel.com/docs'}, {'type': 'enterprise page', 'url': 'https://vercel.com/enterprise'}, {'type': 'events page', 'url': 'https://vercel.com/events'}, {'type': 'features page', 'url': 'https://vercel.com/features'}, {'type': 'guide page', 'url': 'https://vercel.com/guides'}, {'type': 'legal/privacy policy page', 'url': 'https://vercel.com/legal Privacy-Policy'}, {'type': 'marketplace page', 'url': 'https://vercel.com/marketplace'}, {'type': 'partners page', 'url': 'https://vercel.com/partners'}, {'type': 'press page', 'url': 'https://vercel.com/press'}, {'type': 'pricing p

It appears that you've collected various blog posts from Vercel's blog discussing AI, serverless computing, security, and other relevant topics. The posts cover a wide range of subjects, including:

1. **AI-related topics**:
	* Building secure AI agents
	* Introducing the v0 composite model family
	* AI agent development
2. **Serverless computing**:
	* Fluid compute: Evolving serverless for AI workloads
3. **Security and_bot defenses**:
	* Vercel security roundup: improved bot defenses, DoS mitigations, and insights
4. **Storage**:
	* Vercel Blob is now generally available
5. **General updates**:
	* Building efficient MCP servers (Model Context Protocol)
	* Designing the conference platform for Vercel Ship 2025

If you're looking to deploy an AI-powered application on Vercel, I'd recommend exploring their resources and blog posts on topics like:

* Building secure AI agents
* Introducing the v0 composite model family
* Model Context Protocol (MCP) server creation and integration with various frameworks
* Using Vercel's Blob storage for cost-efficient and durable storage

Feel free to ask if you need further assistance or guidance!

In [81]:
stream_brochure_ollama("Vercel", "https://vercel.com/", "llama3.2")

Found links: {'links': [{'type': 'about', 'url': 'https://vercel.com/about'}, {'type': 'products/rendering', 'url': 'https://vercel.com/products/rendering'}, {'type': 'products/observability', 'url': 'https://vercel.com/products/observability'}, {'type': 'security', 'url': 'https://vercel.com/security'}, {'type': 'frameworks/nextjs', 'url': 'https://vercel.com/frameworks/nextjs'}, {'type': 'solutions/turborepo', 'url': 'https://vercel.com/solutions/turborepo'}, {'type': 'sdk', 'url': 'https://sdk.vercel.ai/'}, {'type': 'products/managed-infrastructure', 'url': 'https://vercel.com/products/managed-infrastructure'}, {'type': 'enterprise', 'url': 'https://vercel.com/enterprise'}, {'type': 'features/edge-functions', 'url': 'https://vercel.com/features/edge-functions'}, {'type': 'blog', 'url': 'https://vercel.com/blog'}, {'type': 'changelog', 'url': 'https://vercel.com/changelog'}, {'type': 'press', 'url': 'https://vercel.com/press'}, {'type': 'events', 'url': 'https://vercel.com/events'}, 

This appears to be a comprehensive documentation website for Vercel, a platform that enables users to build, deploy, and manage modern web applications.

The content is divided into several sections, including:

* **Products**: A list of the different products offered by Vercel, such as DX Platform, AI, Fluid Compute, Next.js, and more.
* **Observability**: Documentation around Vercel's Observability feature, which provides real-time infrastructure and traffic insights to help developers debug and optimize their applications.
* **Logs**: Overview of Vercel's logging features, including the ability to view, search, and inspect logs, as well as send data to favorite logging providers.
* **Web Analytics**: Documentation around Vercel's Web Analytics feature, which provides comprehensive insights into website performance, traffic, and user behavior.

The content is organized in a clear and structured manner, with each section featuring a list of key features, a screenshot or diagram to illustrate the concept, and example code snippets or use cases to demonstrate how to integrate the feature into an application.

Some specific pages of interest include:

* **Observability**: This page provides an overview of Vercel's Observability feature, including its benefits, features, and use cases.
* **Logs**: This page explains how to view, search, and inspect logs in Vercel, as well as send data to favorite logging providers.
* **Web Analytics**: This page describes how to understand visitor engagement, track custom events, and get detailed views of website performance metrics.

Overall, this documentation website seems designed to help developers quickly find the information they need to build, deploy, and optimize their applications on Vercel.